In [1]:
import os
import sys
sys.argv
import argparse
import numpy as np
import cv2

import math
import dlib
from PIL import Image,ImageFile

__version__ = '0.3.0'

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
!pip install patool
import patoolib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 KB 6.2 MB/s eta 0:00:00


In [18]:
patoolib.extract_archive('Dataset_svt.zip')

patool: Extracting Dataset_svt.zip ...
patool: running /usr/bin/7z x -o./Unpack_v3es26zg -- Dataset_svt.zip
patool: ... Dataset_svt.zip extracted to `Dataset_svt' (multiple files in root).


'Dataset_svt'

In [5]:
patoolib.extract_archive('wenjunhui.zip')

patool: Extracting wenjunhui.zip ...
patool: running /usr/bin/7z x -o./Unpack_yv8pbj0s -- wenjunhui.zip
patool: ... wenjunhui.zip extracted to `wenjunhui'.


'wenjunhui'

In [6]:
!git clone https://github.com/X-zhangyang/Real-World-Masked-Face-Dataset.git

Cloning into 'Real-World-Masked-Face-Dataset'...
remote: Enumerating objects: 1417, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 1417 (delta 7), reused 1 (delta 0), pack-reused 1401
Receiving objects: 100% (1417/1417), 180.93 MiB | 23.15 MiB/s, done.
Resolving deltas: 100% (160/160), done.


In [7]:
!git clone https://github.com/ageitgey/face_recognition_models.git

Cloning into 'face_recognition_models'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 57 (delta 8), reused 9 (delta 4), pack-reused 41
Unpacking objects: 100% (57/57), done.
Checking out files: 100% (16/16), done.


In [8]:
!pip install face_recognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566185 sha256=af8d2b4d9a6f078ac1c7cc2082873fb3b66140e971c9f20dbc08e83a37af9377
  Stored in directory: /root/.cache/pip/wheels/b4/4b/8f/751e99d45f089bdf366a7d3e5066db3c2b84a62e4377f534d7
Successfully built face-recognition-models


In [9]:
!pip install cmake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
IMAGE_DIR = os.path.join(os.path.dirname(os.path.abspath("__file__")), '/content/Real-World-Masked-Face-Dataset/wear_mask_to_face/images')

DEFAULT_IMAGE_PATH = os.path.join(IMAGE_DIR, '/content/Real-World-Masked-Face-Dataset/wear_mask_to_face/images/default-mask.png')
BLACK_IMAGE_PATH =  os.path.join(IMAGE_DIR, '/content/Real-World-Masked-Face-Dataset/wear_mask_to_face/images/black-mask.png')
BLUE_IMAGE_PATH =  os.path.join(IMAGE_DIR, '/content/Real-World-Masked-Face-Dataset/wear_mask_to_face/images/blue-mask.png')
RED_IMAGE_PATH =  os.path.join(IMAGE_DIR, '/content/Real-World-Masked-Face-Dataset/wear_mask_to_face/images/red-mask.png')


def rect_to_bbox(rect):
    # print(rect)
    x = rect[3]
    y = rect[0]
    w = rect[1] - x
    h = rect[2] - y
    return (x, y, w, h)


def face_alignment(faces):
    #Predict Key Points
    predictor = dlib.shape_predictor('/content/face_recognition_models/face_recognition_models/models/shape_predictor_68_face_landmarks.dat')
    faces_aligned = []
    for face in faces:
        rec = dlib.rectangle(0, 0, face.shape[0], face.shape[1])
        shape = predictor(np.uint8(face), rec)
        # left eye, right eye, nose, left mouth, right mouth
        order = [36, 45, 30, 48, 54]
        for j in order:
            x = shape.part(j).x
            y = shape.part(j).y
        # Calculate the center coordinates of the two eyes
        eye_center = ((shape.part(36).x + shape.part(45).x) * 1. / 2, (shape.part(36).y + shape.part(45).y) * 1. / 2)
        dx = (shape.part(45).x - shape.part(36).x)
        dy = (shape.part(45).y - shape.part(36).y)
        # calculate angle
        angle = math.atan2(dy, dx) * 180. / math.pi
        # Compute the affine matrix
        RotateMatrix = cv2.getRotationMatrix2D(eye_center, angle, scale=1)
        # Perform an affine transformation, that is, rotate
        RotImg = cv2.warpAffine(face, RotateMatrix, (face.shape[0], face.shape[1]))
        faces_aligned.append(RotImg)
    return faces_aligned

In [11]:
def cli(pic_path, save_pic_path):
    parser = argparse.ArgumentParser(description='Wear a face mask in the given picture.')
    
    parser.add_argument('--model', default='hog', choices=['hog', 'cnn'], help='Which face detection model to use.')
    group = parser.add_mutually_exclusive_group()
    group.add_argument('--black', action='store_true', help='Wear black mask')
    group.add_argument('--blue', action='store_true', help='Wear blue mask')
    group.add_argument('--red', action='store_true', help='Wear red mask')
    parser.add_argument("-f", required=False)
    args = parser.parse_args()

    if not os.path.exists(pic_path):
        print(f'Picture {pic_path} not exists.')
        sys.exit(1)

    #if args.black:
          #mask_path = BLACK_IMAGE_PATH
    #elif args.blue:
          #mask_path = BLUE_IMAGE_PATH
    #elif args.red:
          #mask_path = RED_IMAGE_PATH
    #else:
          #mask_path = DEFAULT_IMAGE_PATH
    mask_path = DEFAULT_IMAGE_PATH
    print(pic_path,mask_path)
    print(save_pic_path)
    unmasked_paths = FaceMasker(pic_path, mask_path, True, 'cnn',save_pic_path).mask()
    return unmasked_paths

In [12]:
class FaceMasker:
    KEY_FACIAL_FEATURES = ('nose_bridge', 'chin')

    def __init__(self, face_path, mask_path, show=False, model='cnn',save_path = ''):
        self.face_path = face_path
        self.mask_path = mask_path
        self.save_path = save_path
        self.show = show
        self.model = model
        self._face_img: ImageFile = None
        self._mask_img: ImageFile = None

    def mask(self):
        import face_recognition

        face_image_np = face_recognition.load_image_file(self.face_path)
        face_locations = face_recognition.face_locations(face_image_np, model=self.model)
        face_landmarks = face_recognition.face_landmarks(face_image_np, face_locations)
        self._face_img = Image.fromarray(face_image_np)
        self._mask_img = Image.open(self.mask_path)

        found_face = False
        for face_landmark in face_landmarks:
            # check whether facial features meet requirement
            skip = False
            for facial_feature in self.KEY_FACIAL_FEATURES:
                if facial_feature not in face_landmark:
                    skip = True
                    break
            if skip:
                continue

            # mask face
            found_face = True
            self._mask_face(face_landmark)

        unmasked_paths = []
        if found_face:
            # align
            src_faces = []
            src_face_num = 0
            with_mask_face = np.asarray(self._face_img)
            for (i, rect) in enumerate(face_locations):
                src_face_num = src_face_num + 1
                (x, y, w, h) = rect_to_bbox(rect)
                detect_face = with_mask_face[y:y + h, x:x + w]
                src_faces.append(detect_face)
            # Face alignment operation and save
            faces_aligned = face_alignment(src_faces)
            face_num = 0
            for faces in faces_aligned:
                face_num = face_num + 1
                faces = cv2.cvtColor(faces, cv2.COLOR_RGBA2BGR)
                size = (int(128), int(128))
                faces_after_resize = cv2.resize(faces, size, interpolation=cv2.INTER_AREA)
                cv2.imwrite('/content/savedata'+str(face_num)+'.jpg', faces_after_resize)
            # if self.show:
            #     self._face_img.show()
            # save
            # self._save()
        else:
            #Record the uncropped picture here
            print('Found no face.'+self.save_path)
            unmasked_paths.append(self.save_path)
            return unmasked_paths
            
    def _mask_face(self, face_landmark: dict):
        nose_bridge = face_landmark['nose_bridge']
        nose_point = nose_bridge[len(nose_bridge) * 1 // 4]
        nose_v = np.array(nose_point)

        chin = face_landmark['chin']
        chin_len = len(chin)
        chin_bottom_point = chin[chin_len // 2]
        chin_bottom_v = np.array(chin_bottom_point)
        chin_left_point = chin[chin_len // 8]
        chin_right_point = chin[chin_len * 7 // 8]

        # split mask and resize
        width = self._mask_img.width
        height = self._mask_img.height
        width_ratio = 1.2
        new_height = int(np.linalg.norm(nose_v - chin_bottom_v))

        # left
        mask_left_img = self._mask_img.crop((0, 0, width // 2, height))
        mask_left_width = self.get_distance_from_point_to_line(chin_left_point, nose_point, chin_bottom_point)
        mask_left_width = int(mask_left_width * width_ratio)
        mask_left_img = mask_left_img.resize((mask_left_width, new_height))

        # right
        mask_right_img = self._mask_img.crop((width // 2, 0, width, height))
        mask_right_width = self.get_distance_from_point_to_line(chin_right_point, nose_point, chin_bottom_point)
        mask_right_width = int(mask_right_width * width_ratio)
        mask_right_img = mask_right_img.resize((mask_right_width, new_height))

        # merge mask
        size = (mask_left_img.width + mask_right_img.width, new_height)
        mask_img = Image.new('RGBA', size)
        mask_img.paste(mask_left_img, (0, 0), mask_left_img)
        mask_img.paste(mask_right_img, (mask_left_img.width, 0), mask_right_img)

        # rotate mask
        angle = np.arctan2(chin_bottom_point[1] - nose_point[1], chin_bottom_point[0] - nose_point[0])
        rotated_mask_img = mask_img.rotate(angle, expand=True)

        # calculate mask location
        center_x = (nose_point[0] + chin_bottom_point[0]) // 2
        center_y = (nose_point[1] + chin_bottom_point[1]) // 2

        offset = mask_img.width // 2 - mask_left_img.width
        radian = angle * np.pi / 180
        box_x = center_x + int(offset * np.cos(radian)) - rotated_mask_img.width // 2
        box_y = center_y + int(offset * np.sin(radian)) - rotated_mask_img.height // 2

        # add mask
        self._face_img.paste(mask_img, (box_x, box_y), mask_img)

    def _save(self):
        path_splits = os.path.splitext(self.face_path)
        new_face_path = path_splits[0] + '-with-mask' + path_splits[1]
        self._face_img.save(new_face_path)
        print(f'Save to {new_face_path}')

    @staticmethod
    def get_distance_from_point_to_line(point, line_point1, line_point2):
        distance = np.abs((line_point2[1] - line_point1[1]) * point[0] +
                          (line_point1[0] - line_point2[0]) * point[1] +
                          (line_point2[0] - line_point1[0]) * line_point1[1] +
                          (line_point1[1] - line_point2[1]) * line_point1[0]) / \
                   np.sqrt((line_point2[1] - line_point1[1]) * (line_point2[1] - line_point1[1]) +
                           (line_point1[0] - line_point2[0]) * (line_point1[0] - line_point2[0]))
        return int(distance)

In [30]:
if __name__ == "__main__":
    #dataset_path = '/content/tes'
    save_dataset_path = '/content/savedata'
    imgpath = '/content/Dataset_svt/dino.jpg'
    cli(imgpath, save_dataset_path)
    #for root, dirs, files in os.walk(dataset_path, topdown=False):
        #for name in files:
            #new_root = root.replace(dataset_path, save_dataset_path)
            # if not os.path.exists(new_root):
            #     os.makedirs(new_root)
            # deal
            #imgpath = os.path.join(root, name)
            #save_imgpath = os.path.join(new_root, name)
            #cli(imgpath,save_imgpath)

/content/Dataset_svt/dino.jpg /content/Real-World-Masked-Face-Dataset/wear_mask_to_face/images/default-mask.png
/content/savedata
Found no face./content/savedata
